<a href="https://colab.research.google.com/github/tajbiahossain/DNA-Sequence-Classification/blob/main/DNA_seq_data_H3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import re
import nltk
import pandas as pd
from textblob import Word
import numpy as np
import csv
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.layers import  Dropout, Dense
from sklearn import metrics

In [13]:
from google.colab import files
uploaded = files.upload()

Saving dataset01.txt to dataset01.txt


In [14]:
data = pd.read_csv('dataset01.txt', sep = '\n')
print(data.head())

                                            all_data
0                                     >iTELL-Chr1_61
1  CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
2                                                  0
3                                   >iYAL067W-A_5085
4  AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...


In [15]:
all=data["all_data"].tolist()
cls=[]
sequence=[]
label=[]
all

['>iTELL-Chr1_61',
 'CACACCACACCCACACACCCACACACCACACCACACACCACACCACACCCACACACACACATCCTAACACTACCCTAACACAGCCCTAATCTAACCCTGGCCAACCTGTCTCTCAACTTACCCTCCATTACCCTGCCTCCACTCGTTACCCTGTCCCATTCAACCATACCACTCCGAACCACCATCCATCCCTCTACTTACTACCACTCACCCACCGTTACCCTCCAATTACCCATATCCAACCCACTGCCACTTACCCTACCATTACCCTACCATCCACCATGACCTACTCACCATACTGTTCTTCTAC',
 '0',
 '>iYAL067W-A_5085',
 'AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCTTGTGCTTTCTTAGTATCTATCACAAAGGAGACAAAATCGTTGATAAAAAGTGCATCAACATTCCCAGCCAGAAAATGCACATCATAAAGACATGTTATTCAAGAGCCACGACCGTCTTCAATTTATCTTTTATAAAAAACCCTTGTTCTACTGACAGGATGGAATAGATATTAAATATACATTTTGCATTTTTTTTTTTTTCTGTATTGAAGATTTGTATATGAAAGATGTTTATACATCAAATGCTTTGAATAAAGCCATCTTAATTTCAATTTCATGCCCTCCTTCACCGTTTTCTGTTGGTCTAGAGGTAGCTTGTTGTGGTCACTAATGAGAACTTAAATAGTTTTCAACTGCTGGTGGTAAATCAATAATTTATGTTCTTAACCTAACATTTGATGACCTTTGATGCGTTGGTTATGTTGAAGACAAATTGCCTCTAATCAGTTCCATT',
 '0',
 '>iYAL067W-A_6786',
 'AATTATATTTCCATCAGCTCAATACCGCAGTACTTTGAAACCTGATTTATATATTGCAGAACTTAATTAAAAGTACATTGTAGTTCAAAAAATAAATATCAAACT

In [16]:
length = len(all)
for i in range(length):
   if (i%3)==0:
             cls.append(all[i])
   else:
      if(all[i]== "0" or all[i]=="1"):
        label.append(all[i])
      else:
         sequence.append(all[i])
        
     


In [17]:
print(len(cls))
print(len(label))
print(len(sequence))

14965
14965
14965


In [18]:

df = pd.DataFrame(list(zip(cls, label, sequence)),
               columns =['class' , 'label', 'sequence'])
df.head(10)


,class,label,sequence
0,>iTELL-Chr1_61,0,CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
1,>iYAL067W-A_5085,0,AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...
2,>iYAL067W-A_6786,0,AATTATATTTCCATCAGCTCAATACCGCAGTACTTTGAAACCTGAT...
3,>iYAL067W-A_7060,0,AACAATAGTGGGTATGAGTAAAGATATATAGATCGATATTTTGAAT...
4,>YAL067C_SEO1_8061,1,CAAAGATTTCAACCATAGTAGGTATGCCCCAGATGAAACATTACTG...
5,>YAL067C_SEO1_8340,1,CACAATAGCGTCAATAATAAAGTTCCATCTCCATCCCTCTAAACCA...
6,>iYAL067C_9509,1,GTTTTTTTCCAGTGACACAATCTTTACCATTACACAGTTTTTACTA...
7,>iYAL067C_10442,0,CAGAAACATAAAATTATATCACTTTATTTCATATGGTTTCATGCTT...
8,>YAL065C_YAL065C_11570,1,AGATACCTCGAAACAAACTCTATGTAAACACTTATTTTATTGTGGT...
9,>iYAL064W-B_13286,1,ATGACTGGAGAAGAAGTGACTAGAGAAGAAGTGACTGAGGAAGAAA...


In [19]:
# saving the dataframe
df.to_csv('file2.csv', header=False, index=False)

In [20]:
df.head()

,class,label,sequence
0,>iTELL-Chr1_61,0,CACACCACACCCACACACCCACACACCACACCACACACCACACCAC...
1,>iYAL067W-A_5085,0,AATTTTTATAGGTCGACCCTTCTGTCGCTTACTGGGTTGATTATCT...
2,>iYAL067W-A_6786,0,AATTATATTTCCATCAGCTCAATACCGCAGTACTTTGAAACCTGAT...
3,>iYAL067W-A_7060,0,AACAATAGTGGGTATGAGTAAAGATATATAGATCGATATTTTGAAT...
4,>YAL067C_SEO1_8061,1,CAAAGATTTCAACCATAGTAGGTATGCCCCAGATGAAACATTACTG...


In [29]:
import pydna as PyDNA
from sklearn.preprocessing import OneHotEncoder


In [32]:
dna_sequence = df['sequence']
dna_np_array = dna_sequence.to_numpy()
dna_np_array=dna_np_array.reshape(-1,1)
encoder = OneHotEncoder()
# transform data
onehot = encoder.fit_transform(dna_np_array)
print(onehot)


  (0, 4947)	1.0
  (1, 1619)	1.0
  (2, 1510)	1.0
  (3, 614)	1.0
  (4, 4649)	1.0
  (5, 4939)	1.0
  (6, 10381)	1.0
  (7, 5093)	1.0
  (8, 2667)	1.0
  (9, 3941)	1.0
  (10, 70)	1.0
  (11, 13296)	1.0
  (12, 12034)	1.0
  (13, 9108)	1.0
  (14, 12184)	1.0
  (15, 10549)	1.0
  (16, 11654)	1.0
  (17, 11096)	1.0
  (18, 12119)	1.0
  (19, 1754)	1.0
  (20, 11157)	1.0
  (21, 11395)	1.0
  (22, 9371)	1.0
  (23, 8475)	1.0
  (24, 12405)	1.0
  :	:
  (14940, 11081)	1.0
  (14941, 4232)	1.0
  (14942, 2127)	1.0
  (14943, 5424)	1.0
  (14944, 5196)	1.0
  (14945, 9307)	1.0
  (14946, 12910)	1.0
  (14947, 6557)	1.0
  (14948, 285)	1.0
  (14949, 4520)	1.0
  (14950, 7853)	1.0
  (14951, 9685)	1.0
  (14952, 8681)	1.0
  (14953, 4206)	1.0
  (14954, 1200)	1.0
  (14955, 1739)	1.0
  (14956, 908)	1.0
  (14957, 3325)	1.0
  (14958, 13903)	1.0
  (14959, 11270)	1.0
  (14960, 11652)	1.0
  (14961, 3597)	1.0
  (14962, 4255)	1.0
  (14963, 11086)	1.0
  (14964, 14117)	1.0


In [36]:
label = df['label']
label_array = label.to_numpy()
label_array=label_array.reshape(-1,1)
encoder = OneHotEncoder()
# transform data
onehot = encoder.fit_transform(label_array)
print(onehot)

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 1)	1.0
  (5, 1)	1.0
  (6, 1)	1.0
  (7, 0)	1.0
  (8, 1)	1.0
  (9, 1)	1.0
  (10, 0)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 0)	1.0
  (15, 0)	1.0
  (16, 0)	1.0
  (17, 0)	1.0
  (18, 0)	1.0
  (19, 1)	1.0
  (20, 1)	1.0
  (21, 1)	1.0
  (22, 1)	1.0
  (23, 1)	1.0
  (24, 1)	1.0
  :	:
  (14940, 0)	1.0
  (14941, 1)	1.0
  (14942, 0)	1.0
  (14943, 0)	1.0
  (14944, 1)	1.0
  (14945, 1)	1.0
  (14946, 1)	1.0
  (14947, 0)	1.0
  (14948, 0)	1.0
  (14949, 0)	1.0
  (14950, 0)	1.0
  (14951, 0)	1.0
  (14952, 0)	1.0
  (14953, 0)	1.0
  (14954, 0)	1.0
  (14955, 1)	1.0
  (14956, 0)	1.0
  (14957, 0)	1.0
  (14958, 0)	1.0
  (14959, 0)	1.0
  (14960, 0)	1.0
  (14961, 1)	1.0
  (14962, 1)	1.0
  (14963, 1)	1.0
  (14964, 1)	1.0
